# NB1: assembly Pedicularis RAD data


### Required software

In [1]:
# conda install ipyrad -c ipyrad
import ipyrad as ip


### The demultiplexed data files. 
See notebook 0 to see demultiplexing strategy. We now have xxx files in three directories for the three libraries. 

In [4]:
data1 = ip.Assembly("lib1")
data1.params.project_dir = "../analysis-ipyrad"
data1.params.sorted_fastq_path = "~/Documents/RADSEQ_DATA/demux_Pedicularis/demux1_fastqs/*.gz"


New Assembly: lib1


In [5]:
data2 = ip.Assembly("lib2")
data2.params.project_dir = "../analysis-ipyrad"
data2.params.sorted_fastq_path = "~/Documents/RADSEQ_DATA/demux_Pedicularis/demux2_fastqs/*.gz"


New Assembly: lib2


In [6]:
data3 = ip.Assembly("lib3")
data3.params.project_dir = "../analysis-ipyrad"
data3.params.sorted_fastq_path = "~/Documents/RADSEQ_DATA/demux_Pedicularis/demux3_fastqs/*.gz"


New Assembly: lib3


In [13]:
#ll /home/deren/Documents/RADSEQ_DATA/demux_Pedicularis/demux1_fastqs/
data1.run("1", force=True, auto=True)

Parallel connection | 0cba0d69e8d2: 40 cores
[####################] 100% 0:00:00 | loading reads        | s1 |


In [14]:
# load demux reads from each location into three separate assembly objects
data1.run("1", force=True, auto=True)
data2.run("1", force=True, auto=True)
data3.run("1", force=True, auto=True)

Parallel connection | 0cba0d69e8d2: 40 cores
[####################] 100% 0:00:00 | loading reads        | s1 |
Parallel connection | 0cba0d69e8d2: 40 cores
[####################] 100% 0:00:00 | loading reads        | s1 |
Parallel connection | 0cba0d69e8d2: 40 cores
[####################] 100% 0:00:00 | loading reads        | s1 |


### Subselecting samples for this study

In [18]:
# load each assembly object above now has a JSON record it could be re-loaded from
data1 = ip.load_json("../analysis-ipyrad/lib1.json")
data2 = ip.load_json("../analysis-ipyrad/lib2.json")
data3 = ip.load_json("../analysis-ipyrad/lib3.json")

loading Assembly: lib1
from saved path: ~/Documents/community-character-displacement/analysis-ipyrad/lib1.json
loading Assembly: lib2
from saved path: ~/Documents/community-character-displacement/analysis-ipyrad/lib2.json
loading Assembly: lib3
from saved path: ~/Documents/community-character-displacement/analysis-ipyrad/lib3.json


In [19]:
# combine all samples into one assembly named "alldata"
alldata = ip.merge('alldata', [data1, data2, data3])

# print N samples in merged assembly
print("{} total samples".format(len(alldata.samples)))

New Assembly: alldata
191 total samples


In [20]:
# select out all cranolopha samples 
cranodata = [i for i in alldata.samples if 'cranolopha' in i]

# select out four closest outgroups
outgroups = ["30106_fletcheri", "41112_elwesii", "39969_torta", "33286_cephalantha"]

# remove DE_8.2 cranolopha which has almost no data
keep = [i for i in cranodata + outgroups if i != "DE_8.2_cranolopha"]

# make a new branch
subdata = alldata.branch("subdata", subsamples=keep, force=True)

# print N samples
print("{} samples in subsampled cranolopha data set".format(len(subdata.samples)))

114 samples in subsampled cranolopha data set


### Merge technical replicates
In a preliminary analyses the technical replicates of population 12 all placed sister to each other as expected in phylogenetic analyses. Therefore we merge them together in this analyses so that each sample receives by two input fastq files. 

In [21]:
# which ones have replicate?
[i for i in subdata.samples if "replicate" in i]

['DE_12.2_replicate_cranolopha',
 'DE_12.4_replicate_cranolopha',
 'DE_12.5_replicate_cranolopha',
 'DE_6.8_replicate_cranolopha',
 'DE_12.3_replicate_cranolopha',
 'DE_12.7_replicate_cranolopha',
 'DE_12.6_replicate_cranolopha',
 'DE_12.1_replicate_cranolopha']

In [22]:
# make a dictionary for merging technical replicates {oldname: newname}
mergedict = {
    "DE_12.1_replicate_cranolopha": "DE_12.1_cranolopha",
    "DE_12.2_replicate_cranolopha": "DE_12.2_cranolopha", 
    "DE_12.3_replicate_cranolopha": "DE_12.3_cranolopha", 
    "DE_12.4_replicate_cranolopha": "DE_12.4_cranolopha", 
    "DE_12.5_replicate_cranolopha": "DE_12.5_cranolopha", 
    "DE_12.6_replicate_cranolopha": "DE_12.6_cranolopha",
    "DE_12.7_replicate_cranolopha": "DE_12.7_cranolopha",
    "DE_6.8_replicate_cranolopha": "DE_6.8_cranolopha", 
}

In [23]:
# reduce by merging technical replicates together
cranos = ip.merge("cranos", subdata, mergedict)

# print N samples
print("{} samples in subsampled and merged cranolopha data set".format(len(cranos.samples)))

New Assembly: cranos
106 samples in subsampled and merged cranolopha data set


## Data Assembly

In [40]:
# set some initial parameters
cranos.params.project_dir = "../analysis-ipyrad"
cranos.params.filter_adapters = 3
cranos.params.phred_Qscore_offset = 43                                           
cranos.params.clust_threshold = 0.90
cranos.params.output_formats = "pslvk"

# save json to project dir
cranos.save()

# show parameters
cranos.params

0   assembly_name               cranos                                       
1   project_dir                 ~/Documents/community-character-displacement/analysis-ipyrad
2   raw_fastq_path              Merged: subdata                              
3   barcodes_path               Merged: subdata                              
4   sorted_fastq_path           Merged: subdata                              
5   assembly_method             denovo                                       
6   reference_sequence                                                       
7   datatype                    rad                                          
8   restriction_overhang        ('TGCAG', '')                                
9   max_low_qual_bases          5                                            
10  phred_Qscore_offset         43                                           
11  mindepth_statistical        6                                            
12  mindepth_majrule            6                

### Assemble to full loci database

In [3]:
cranos = ip.load_json("../analysis-ipyrad/cranos.json")

loading Assembly: cranos
from saved path: ~/Documents/community-character-displacement/analysis-ipyrad/cranos.json


In [ ]:
cranos.run("23456", auto=True, force=True)

Parallel connection | 0cba0d69e8d2: 40 cores
[####################] 100% 0:00:24 | concatenating inputs | s2 |
[####################] 100% 0:23:18 | processing reads     | s2 |
[####################] 100% 0:00:05 | concatenating        | s3 |
[####################] 100% 0:01:33 | dereplicating        | s3 |
[                    ]   0% 0:11:12 | clustering/mapping   | s3 |

### Filter to final data sets

In [3]:
# this will be used for phylogenetic inference
min50 = cranos.branch("cranos_min50")
min50.params.min_samples_locus = 50
min50.run("7", auto=True, force=True)

Parallel connection | 0cba0d69e8d2: 40 cores
[####################] 100% 0:00:50 | applying filters     | s7 |
[####################] 100% 0:01:41 | building arrays      | s7 |
[####################] 100% 0:00:24 | writing conversions  | s7 |
[####################] 100% 0:01:32 | indexing vcf depths  | s7 |
[####################] 100% 0:04:19 | writing vcf output   | s7 |


In [6]:
# make a new branch
pop4 = cranos.branch("cranos_pop4")

# set population minimums
pop4.populations = {
    "out": (1, [i for i in pop4.samples if "crano" not in i]),
    "p1": (4, [i for i in pop4.samples if "DE_1." in i]),
    "p4": (4, [i for i in pop4.samples if "DE_4." in i]),
    "p6": (4, [i for i in pop4.samples if "DE_6." in i]),
    "p8": (4, [i for i in pop4.samples if "DE_8." in i]),
    "p9": (4, [i for i in pop4.samples if "DE_9." in i]),
    "p10": (4, [i for i in pop4.samples if "DE_10." in i]),
    "p11": (4, [i for i in pop4.samples if "DE_11." in i]),
    "p12": (4, [i for i in pop4.samples if "DE_12." in i]),
    "p15": (4, [i for i in pop4.samples if "DE_15." in i]),
    "p18": (4, [i for i in pop4.samples if "DE_18." in i]),
    "p19": (4, [i for i in pop4.samples if "DE_19." in i]),
    "p22": (4, [i for i in pop4.samples if "DE_22." in i]),
    "p23": (4, [i for i in pop4.samples if "DE_23." in i]),
    "p24": (4, [i for i in pop4.samples if "DE_24." in i]),
    "p26": (4, [i for i in pop4.samples if "DE_26." in i]),
}

# run step 7 for population data
pop4.run('7', force=True, auto=True)

Parallel connection | 0cba0d69e8d2: 40 cores
[####################] 100% 0:00:49 | applying filters     | s7 |
[####################] 100% 0:00:42 | building arrays      | s7 |
[####################] 100% 0:00:09 | writing conversions  | s7 |
[####################] 100% 0:00:24 | indexing vcf depths  | s7 |
[####################] 100% 0:01:34 | writing vcf output   | s7 |


### A population min4 with with outgroups excluded

In [8]:
# make a new branch
keep = {}
pop4no = cranos.branch("cranos_pop4_nout", subsamples=keep)

# set population minimums
pop4no.populations = {
    "p1": (4, [i for i in pop4no.samples if "DE_1." in i]),
    "p4": (4, [i for i in pop4no.samples if "DE_4." in i]),
    "p6": (4, [i for i in pop4no.samples if "DE_6." in i]),
    "p8": (4, [i for i in pop4no.samples if "DE_8." in i]),
    "p9": (4, [i for i in pop4no.samples if "DE_9." in i]),
    "p10": (4, [i for i in pop4no.samples if "DE_10." in i]),
    "p11": (4, [i for i in pop4no.samples if "DE_11." in i]),
    "p12": (4, [i for i in pop4no.samples if "DE_12." in i]),
    "p15": (4, [i for i in pop4no.samples if "DE_15." in i]),
    "p18": (4, [i for i in pop4no.samples if "DE_18." in i]),
    "p19": (4, [i for i in pop4no.samples if "DE_19." in i]),
    "p22": (4, [i for i in pop4no.samples if "DE_22." in i]),
    "p23": (4, [i for i in pop4no.samples if "DE_23." in i]),
    "p24": (4, [i for i in pop4no.samples if "DE_24." in i]),
    "p26": (4, [i for i in pop4no.samples if "DE_26." in i]),
}

# run step 7 for population data
pop4no.run('7', force=True, auto=True)

Parallel connection | 0cba0d69e8d2: 40 cores
[####################] 100% 0:00:46 | applying filters     | s7 |
[####################] 100% 0:00:47 | building arrays      | s7 |
[####################] 100% 0:00:10 | writing conversions  | s7 |
[####################] 100% 0:00:25 | indexing vcf depths  | s7 |
[####################] 100% 0:01:39 | writing vcf output   | s7 |


### Link to final stats files hosted on GitHub

+ [crano_min50](https://github.com/eaton-lab/community-character-displacement/blob/master/analysis-ipyrad/cranos_min50_outfiles/cranos_min50_stats.txt)
+ [crano_pop4](https://github.com/eaton-lab/community-character-displacement/blob/master/analysis-ipyrad/cranos_pop4_outfiles/cranos_pop4_stats.txt)
+ [crano_pop4_nout](https://github.com/eaton-lab/community-character-displacement/blob/master/analysis-ipyrad/cranos_pop4_nout_outfiles/cranos_pop4_nout_stats.txt)